In [1]:
import os
import torch
from kandinsky import get_I2V_pipeline
from huggingface_hub import snapshot_download

from IPython.display import Video

### Load pipe 
first, ensure that you have downloaded the models using `download_models.py`

In [ ]:
pipe = get_I2V_pipeline(
    device_map={"dit": "cuda:0", "vae": "cuda:0", "text_embedder": "cuda:0"},
	conf_path="./configs/k5_lite_i2v_5s_sft_sd.yaml", magcache=True
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


using Magcache
load dit on cuda:1


#### Load LoRa checkpoints from HF

Available LoRas for Lite model:

    `Arc-right`

    `Arc-left`

    `Microwave-right`

    `Microwave-left`

    `Dolly-in`

    `Dolly-out`

    `Truck-right`

    `Trcuk-left`

Available LoRas for Pro model:

    `Arc-right`

    `Arc-left`

    `Microwave-right`

    `Microwave-left`

Note: For optimal performance, select an appropriate LoRa for the t2v or i2v modality.

##### Download the adapter from HF:

In [3]:
cache_dir = './weights'
lora_name = 'Microwave-right'
repo_id = f"kandinskylab/Kandinsky-5.0-I2V-Lite-LoRa-{lora_name}"
adapter_path = snapshot_download(
    repo_id=repo_id,
    local_dir=os.path.join(cache_dir, repo_id),
    token=None
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Load the adapter into the pipe:

In [4]:
pipe.load_adapter(
    adapter_config=os.path.join(cache_dir, repo_id, "config_lora.json"),
    adapter_path= os.path.join(cache_dir, repo_id, "lora.safetensors"),
    adapter_name=lora_name
)

Note: The trigger words we use are stored in the LoRA safetensors metadata. They are automatically concatenated with prompts during inference.

In [5]:
pipe.peft_triggers

{'Microwave-right': 'M1CR0W4V3 r0t4tION, 360-degree rotation. '}

Generate:

In [6]:
save_dir = './outputs/loras/'
os.makedirs(save_dir, exist_ok=True)

out = pipe(
    "A bear wearing an ushanka hat and holding a balalaika in a snowy forest is dancing.",
    image = "assets/test_lora.png",
    time_length=5,
    num_steps=50,
    save_path=os.path.join(save_dir, f'i2v_lite_{lora_name}.mp4'),
    seed=12345,
)
Video(os.path.join(save_dir, f'i2v_lite_{lora_name}.mp4'))

100%|██████████| 50/50 [02:20<00:00,  2.80s/it]
/home/user/conda/envs/kandinsky-cuda12.8/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Download another adapter and switch to it:

In [7]:
lora_name = 'Microwave-left'
repo_id = f"kandinskylab/Kandinsky-5.0-I2V-Lite-LoRa-{lora_name}"
adapter_path = snapshot_download(
    repo_id=repo_id,
    local_dir=os.path.join(cache_dir, repo_id),
    token=None
)

pipe.load_adapter(
    adapter_config=os.path.join(cache_dir, repo_id, "config_lora.json"),
    adapter_path= os.path.join(cache_dir, repo_id, "lora.safetensors"),
    adapter_name=lora_name
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
out = pipe(
    "A bear wearing an ushanka hat and holding a balalaika in a snowy forest is dancing.",
    image="assets/test_lora.png",
    time_length=5,
    num_steps=50,
    save_path=os.path.join(save_dir, f'i2v_lite_{lora_name}.mp4'),
    seed=12345,
)
Video(os.path.join(save_dir, f'i2v_lite_{lora_name}.mp4'))

100%|██████████| 50/50 [01:41<00:00,  2.02s/it]
/home/user/conda/envs/kandinsky-cuda12.8/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Switch to the previous loaded LoRa adapter:

In [9]:
prev_lora_name = 'Microwave-right'
pipe.set_adapter(prev_lora_name)

In [10]:
out = pipe(
    "A bear wearing an ushanka hat and holding a balalaika in a snowy forest is dancing.",
    image="assets/test_lora.png",
    time_length=5,
    num_steps=50,
    save_path=os.path.join(save_dir, f'i2v_lite_{prev_lora_name}_v2.mp4'),
    seed=12345,
)
Video(os.path.join(save_dir, f'i2v_lite_{prev_lora_name}_v2.mp4'))

100%|██████████| 50/50 [01:41<00:00,  2.02s/it]
/home/user/conda/envs/kandinsky-cuda12.8/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Disable all adapters:

In [13]:
pipe.disable_adapters()

In [14]:
out = pipe(
    "A bear wearing an ushanka hat and holding a balalaika in a snowy forest is dancing.",
    image="assets/test_lora.png",
    time_length=5,
    num_steps=50,
    save_path=os.path.join(save_dir, f'i2v_lite_default.mp4'),
    seed=12345,
)
Video(os.path.join(save_dir, f'i2v_lite_default.mp4'))

100%|██████████| 50/50 [01:37<00:00,  1.95s/it]
/home/user/conda/envs/kandinsky-cuda12.8/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Set already loaded adapter again:

In [15]:
pipe.set_adapter(prev_lora_name)

In [17]:
out = pipe(
    "A bear wearing an ushanka hat and holding a balalaika in a snowy forest is dancing.",
    image="assets/test_lora.png",
    time_length=5,
    num_steps=50,
    save_path=os.path.join(save_dir, f'i2v_lite_{prev_lora_name}_v3.mp4'),
    seed=12345,
)
Video(os.path.join(save_dir, f'i2v_lite_{prev_lora_name}_v3.mp4'))

100%|██████████| 50/50 [01:40<00:00,  2.01s/it]
